In [2]:
!pip install sodapy

In [3]:
## Contar el total de filas del dataset rpmr-utcd
from sodapy import Socrata

client = Socrata("www.datos.gov.co", "FnfqzXBmapgY0b6XwJwBceYGJ", timeout=60)

# Hacer la consulta de conteo
resultado = client.get("rpmr-utcd", select="count(*)")
print("Total de filas:", resultado[0]["count"])


Total de filas: 19580610


In [ ]:
## Contal el total de filas del dataset nudc-7mev
from sodapy import Socrata

client = Socrata("www.datos.gov.co", "FnfqzXBmapgY0b6XwJwBceYGJ", timeout=60)

# Hacer la consulta de conteo
resultado = client.get("nudc-7mev", select="count(*)")
print("Total de filas:", resultado[0]["count"])

Total de filas: 14585


In [ ]:
from sodapy import Socrata
import pandas as pd
import time
import os

# App Token válido
token = "FnfqzXBmapgY0b6XwJwBceYGJ"

# Cliente Socrata
client = Socrata("www.datos.gov.co", token, timeout=1000)

# Dataset a descargar
datasets = {
    "SECOP": "rpmr-utcd",
    "MEN": "nudc-7mev"
}

# Función para crear carpeta si no existe
def crear_carpeta(nombre):
    if not os.path.exists(nombre):
        os.makedirs(nombre)

# Función para descargar en lotes y guardar por partes
def descargar_en_partes(nombre, dataset_id, limit=50000, max_filas_por_archivo=1_000_000):
    offset = 0
    total_descargadas = 0
    parte = 1
    buffer = []

    carpeta = nombre  # nombre de la carpeta
    crear_carpeta(carpeta)

    print(f"Iniciando descarga de {nombre} ({dataset_id})...")

    while True:
        try:
            print(f"Descargando desde offset {offset}")
            data = client.get(dataset_id, limit=limit, offset=offset)

            if not data:
                print(f"Descarga completa para {nombre}. Total filas: {total_descargadas}")
                break

            buffer.extend(data)
            total_descargadas += len(data)
            offset += limit

            if len(buffer) >= max_filas_por_archivo:
                df = pd.DataFrame.from_records(buffer)
                archivo = os.path.join(carpeta, f"{nombre}_parte{parte}.csv")
                df.to_csv(archivo, index=False)
                print(f"Guardado: {archivo} (filas: {len(df)})")
                buffer.clear()
                parte += 1

            time.sleep(1.5)

        except Exception as e:
            print(f"Error en offset {offset}: {e}")
            break

    # Guardar último lote si quedó algo pendiente
    if buffer:
        df = pd.DataFrame.from_records(buffer)
        archivo = os.path.join(carpeta, f"{nombre}_parte{parte}.csv")
        df.to_csv(archivo, index=False)
        print(f"Guardado final: {archivo} (filas: {len(df)})")

# Ejecutar descarga para cada dataset
for nombre, dataset_id in datasets.items():
    descargar_en_partes(nombre, dataset_id)


Iniciando descarga de SECOP (rpmr-utcd)...
Descargando desde offset 0
Descargando desde offset 50000
Descargando desde offset 100000
Descargando desde offset 150000
Descargando desde offset 200000
Descargando desde offset 250000
Descargando desde offset 300000
Descargando desde offset 350000
Descargando desde offset 400000
Descargando desde offset 450000
Descargando desde offset 500000
Descargando desde offset 550000
Descargando desde offset 600000
Descargando desde offset 650000
Descargando desde offset 700000
Descargando desde offset 750000
Descargando desde offset 800000
Descargando desde offset 850000
Descargando desde offset 900000
Descargando desde offset 950000
Guardado: SECOP/SECOP_parte1.csv (filas: 1000000)
Descargando desde offset 1000000
Descargando desde offset 1050000
Descargando desde offset 1100000
Descargando desde offset 1150000
Descargando desde offset 1200000
Descargando desde offset 1250000
Descargando desde offset 1300000
Descargando desde offset 1350000
Descargan